In [1]:
import pandas as pd
import numpy as np

# librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt

# librerías para crear el modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder  

from sklearn import tree

# para calcular las métricas
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import cohen_kappa_score


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from sklearn.model_selection import GridSearchCV
from sklearn import tree


from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/train1.csv')
df.head()

,Unnamed: 0,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1,1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183
2,2,2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,7.983
3,3,3,1.08,Very Good,G,SI2,63.2,57.0,6.54,6.50,4.12,8.371
4,4,4,0.36,Premium,G,VS1,62.3,59.0,4.50,4.55,2.82,6.588


In [ ]:
X = df.drop(["id",'price'], axis =1)
y = df['price']


X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

In [ ]:
print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())

In [ ]:
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

In [ ]:
# create a regressor object
regressor = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor.fit(X_train, y_train)

In [ ]:
max_features = np.sqrt(len(X_train.columns))
max_features

In [ ]:
print(regressor.tree_.max_depth)

In [ ]:
y_pred_test_dt = regressor.predict(X_test)
y_pred_train_dt = regressor.predict(X_train)

In [ ]:
from sklearn import metrics

In [ ]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [ ]:
dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Decission Tree I")
dt_results1 #muy overfit

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'predictor': X_train.columns,
                             'importancia': regressor.feature_importances_}
                            )
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

# decision tree

In [ ]:
max_features = np.sqrt(len(X_train.columns))
max_features

In [ ]:
param = {
    "max_depth": list(range(1, 36)),
    "min_samples_split": [2, 5, 10, 20],
    "max_features": [1, 2, 3]
}

arbol2 = GridSearchCV(
            estimator=DecisionTreeRegressor(),
            param_grid= param,
            cv=10,
            verbose=0,
            n_jobs = -1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

In [ ]:
arbol2.fit(X_train, y_train)

In [ ]:
best_tree = arbol2.best_estimator_
best_tree

In [ ]:
y_pred_test_dt2 = best_tree.predict(X_test)
y_pred_train_dt2 = best_tree.predict(X_train)

In [ ]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Decision tree II")

In [ ]:
dt_results2

In [ ]:
import pickle
with open('data/2_primer_modelo.pkl', 'wb') as modelo:
        pickle.dump(best_tree, modelo)

## Random Forest

In [ ]:
param = {    "max_depth": list(range(1, 36)),
    "min_samples_split": [2, 5, 10, 20],
    "max_features": [1, 2, 3]}

bosque = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param,
            cv=10,
            verbose=0,
            n_jobs = -1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

In [ ]:
bosque.fit(X_train, y_train)

In [ ]:
bos = bosque.best_estimator_

In [ ]:
y_pred_test_rf = bos.predict(X_test)
y_pred_train_rf = bos.predict(X_train)

In [ ]:
rf_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
rf_results

In [ ]:
import pickle
with open('data/2_segundo_modelo.pkl', 'wb') as modelo:
        pickle.dump(bos, modelo)

## Gradient

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from multiprocessing import cpu_count

In [ ]:
# Define hyperparameter grid
param_grid = {
    "max_depth": [2,3,4],
    "min_samples_split": [50,100,150],
    "max_features": [1,2,3],
    "min_samples_leaf": [50,100,150]
}

# Create Gradient Boosting Regressor object
gb = GradientBoostingRegressor()

# Create GridSearchCV object with 10-fold cross-validation
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, cv=10, n_jobs=cpu_count())

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

In [ ]:
y_pred_gb_test= grid_search.predict(X_test)
y_pred_gb_train= grid_search.predict(X_train)

In [ ]:
rf_results = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient")
rf_results

In [ ]:
with open('data/gradient_modelo2.pkl', 'wb') as modelo:
        pickle.dump(grid_search, modelo)

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param = {"max_depth": [2,3, 4,6,8,10],
         "min_samples_split": [50, 100, 150, 200, 250],
         "max_features": [1,2,3,4], 
         "min_samples_leaf": [50, 100, 150, 200, 250]}

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = GradientBoostingRegressor(),
        param_grid = param,
        n_jobs     = -1,
        verbose    = 1,
        cv         = 10, 
        return_train_score = True
       )

grid.fit(X = X_train, y = y_train)

In [ ]:
y_pred_gb_test= grid.predict(X_test)
y_pred_gb_train= grid.predict(X_train)

In [ ]:
rf_results = metricas(y_test, y_train, y_pred_gb_test, y_pred_gb_train, "Gradient")
rf_results

In [ ]:
with open('data/gradient_modelo3.pkl', 'wb') as modelo:
        pickle.dump(grid_search, modelo)